# Tratamento prévio dos dados - precipitação no Brasil em (mm)

In [1]:
import os
import pandas as pd
import numpy as np
import csv
import tqdm
import datetime

In [2]:
# Pegando todos os arquivos baixados do site do INMET.
pasta = './dadosprecipita'
arquivos_nome = []
for diretorio, subpastas, arquivos in os.walk(pasta):
    for arquivo in arquivos:
        #print(arquivo)
        arquivos_nome.append(arquivo)

In [3]:
print('Quantidade de arquivos para tratamento:')
print(len(arquivos_nome))
print('Os primeiro arquivos tem o nome de:')
arquivos_nome[0:5]

Quantidade de arquivos para tratamento:
616
Os primeiro arquivos tem o nome de:


['dados_A001_M_2000-05-06_2022-01-01.csv',
 'dados_A002_M_2001-05-28_2022-01-01.csv',
 'dados_A003_M_2001-05-24_2022-01-01.csv',
 'dados_A004_M_2001-05-30_2018-06-20.csv',
 'dados_A005_M_2001-06-03_2022-01-01.csv']

In [4]:
dados = []
localidades = []
for i in tqdm.tqdm(arquivos_nome):
    file = open(f'dadosprecipita/{i}')
    #type(file)
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    localidades.append(header[0][6:])
    rows = []
    for row in csvreader:
        rows.append(row)
        dado = pd.DataFrame(rows[10:])
        if len(dado)>2:
            dado.columns = ['ttt']
            dado = dado.ttt.str.split(';', 1, expand=True)
            dado.columns = ['data','valor']
            dado['valor'] = dado.valor.str.replace(';','')
            dados.append(dado)

100%|██████████| 616/616 [02:55<00:00,  3.51it/s]


In [5]:
%%time
df = pd.concat(dados)
df = df.dropna()

Wall time: 59.2 s


In [6]:
%%time
df.columns = ['Data','Precipitação (mm)']
df

Wall time: 19 ms


,Data,Precipitação (mm)
0,2000-05-31,null
1,2000-06-30,0
2,2000-07-31,.2
0,2000-05-31,null
1,2000-06-30,0
...,...,...
41,2021-09-30,null
42,2021-10-31,null
43,2021-11-30,null
44,2021-12-31,null


In [7]:
%%time
df['Precipitação (mm)'] = pd.to_numeric(df['Precipitação (mm)'], errors='coerce')
df = df.dropna()
df

Wall time: 7.03 s


,Data,Precipitação (mm)
1,2000-06-30,0.0
2,2000-07-31,0.2
1,2000-06-30,0.0
2,2000-07-31,0.2
3,2000-08-31,52.6
...,...,...
19,2019-11-30,70.0
20,2019-12-31,271.0
21,2020-01-31,195.0
22,2020-02-29,367.2


In [8]:
%%time
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
df['Mês'] = df['Data'].dt.month
df['Ano'] = df['Data'].dt.year
df

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 3.52 s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Data,Precipitação (mm),Mês,Ano
1,2000-06-30,0.0,6,2000
2,2000-07-31,0.2,7,2000
1,2000-06-30,0.0,6,2000
2,2000-07-31,0.2,7,2000
3,2000-08-31,52.6,8,2000
...,...,...,...,...
19,2019-11-30,70.0,11,2019
20,2019-12-31,271.0,12,2019
21,2020-01-31,195.0,1,2020
22,2020-02-29,367.2,2,2020


In [9]:
resumo = df.groupby(['Ano','Mês']).agg({'Precipitação (mm)':['mean','median','max','min']})
resumo = resumo.round(2)
resumo = resumo.reset_index()
resumo

Ano Mês Precipitação (mm)                     
                           mean median    max    min
0    2000   6             16.87    2.6   48.0    0.0
1    2000   7             39.13   40.2   77.0    0.2
2    2000   8             89.20   89.2  125.8   52.6
3    2000   9            191.80  191.8  191.8  191.8
4    2000  10            124.00  125.8  189.2   57.0
..    ...  ..               ...    ...    ...    ...
251  2021   8             37.10   20.0  309.0    0.0
252  2021   9             62.18   29.8  279.0    0.0
253  2021  10            160.58  153.3  430.4    0.0
254  2021  11            163.22  153.2  518.6    0.0
255  2021  12            212.87  201.7  785.0    0.0

[256 rows x 6 columns]

In [13]:
resumo.to_excel('resumo_precipitação.xlsx')
resumo.to_csv('resumo_precipitação.csv', index=False)